## Assignment 7

In [18]:
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import pickle
import googlemaps
import json
import pprint
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
import urllib.request, json, csv
import unicodedata

### Import data about geolocated health establishments in Cajamarca

In [20]:
table1 = pd.read_excel('cajamarca_data_helth_estb.xlsx')
table1

,id_eess,codigo_renaes,categoria,nombre,diresa,red,direccion,longitud,latitud,id_ubigeo,...,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,CNT_CCPP,DESCRIPCIO
0,9439,4210,II-1,GRAL. JAEN,CAJAMARCA,NO PERTENECE A NINGUNA RED,AVENIDA AV. PAKAMAUROS S/N S/N AV. PAKAMAUROS ...,-78.804711,-5.706214,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
1,27242,4211,I-4,MORRO SOLAR,CAJAMARCA,JAEN,OTROS ALFREDO BASTOS Nº 630 ALFREDO BASTOS Nº ...,-78.811920,-5.710600,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
2,22911,4212,I-3,MAGLLANAL,CAJAMARCA,JAEN,CALLE CALLE LAS ALMENDRA CDRA. 4 S/N S/N CALLE...,-78.817266,-5.707647,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
3,9242,4213,I-3,FILA ALTA,CAJAMARCA,JAEN,OTROS COMUNIDAD FILA ALTA COMUNIDAD FILA ALTA ...,-78.791643,-5.739184,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
4,26418,4214,I-1,LAS NARANJAS,CAJAMARCA,JAEN,OTROS COMUNIDAD DE LAS NARANJAS COMUNIDAD DE L...,-78.846347,-5.745295,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,21625,5038,I-3,SAN LUIS DE LA LUCMA,CAJAMARCA,SOCOTA,CARRETERA CARRETERA LUCMA SOCOTA DISTRITO SAN ...,-78.607471,-6.292044,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA
1188,8179,6853,I-1,SAIREPAMPA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SAIREPAMPA DISTRITO SAN LUI...,-78.631080,-6.279960,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA
1189,16451,6870,I-1,SANTA ROSA DEL TINGO,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SANTA ROSA DEL TINGO DISTRI...,-78.597640,-6.266560,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA
1190,3040,7048,I-1,SEXESHITA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SEXESHITA DISTRITO SAN LUIS...,-78.627210,-6.267210,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA


In [6]:
table1['nombre'].unique()

array(['GRAL. JAEN', 'MORRO SOLAR', 'MAGLLANAL', ...,
       'SANTA ROSA DEL TINGO', 'SEXESHITA', 'PISAGUAS'], dtype=object)

### Import distric's centroids data (latitude and longitude)

In [21]:
table2 = pd.read_excel('peru_districts_centroids.xlsx')
table2

,CCDD,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,Centroid_Latitude,Centroid_Longitude
0,6,CAJAMARCA,5,CONTUMAZA,4,GUZMANGO,GUZMANGO,60504,605,60504,-7.364304,-78.917689
1,6,CAJAMARCA,11,SAN MIGUEL,5,EL PRADO,EL PRADO,61105,611,61105,-7.038039,-79.003318
2,6,CAJAMARCA,11,SAN MIGUEL,9,NIEPOS,NIEPOS,61109,611,61109,-6.926718,-79.131435
3,6,CAJAMARCA,11,SAN MIGUEL,10,SAN GREGORIO,SAN GREGORIO,61110,611,61110,-7.117978,-79.160968
4,6,CAJAMARCA,12,SAN PABLO,3,SAN LUIS,SAN LUIS GRANDE,61203,612,61203,-7.159046,-78.873084
...,...,...,...,...,...,...,...,...,...,...,...,...
1869,4,AREQUIPA,4,CASTILLA,4,CHACHAS,CHACHAS,40404,404,40404,-15.181923,-72.116054
1870,4,AREQUIPA,4,CASTILLA,5,CHILCAYMARCA,CHILCAYMARCA,40405,404,40405,-15.350613,-72.385276
1871,4,AREQUIPA,4,CASTILLA,6,CHOCO,CHOCO,40406,404,40406,-15.414004,-72.068684
1872,4,AREQUIPA,4,CASTILLA,7,HUANCARQUI,HUANCARQUI,40407,404,40407,-16.012364,-72.368167


In [22]:
table3 = table2[table2.NOMBDEP == 'CAJAMARCA']
table3

,CCDD,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,Centroid_Latitude,Centroid_Longitude
0,6,CAJAMARCA,5,CONTUMAZA,4,GUZMANGO,GUZMANGO,60504,605,60504,-7.364304,-78.917689
1,6,CAJAMARCA,11,SAN MIGUEL,5,EL PRADO,EL PRADO,61105,611,61105,-7.038039,-79.003318
2,6,CAJAMARCA,11,SAN MIGUEL,9,NIEPOS,NIEPOS,61109,611,61109,-6.926718,-79.131435
3,6,CAJAMARCA,11,SAN MIGUEL,10,SAN GREGORIO,SAN GREGORIO,61110,611,61110,-7.117978,-79.160968
4,6,CAJAMARCA,12,SAN PABLO,3,SAN LUIS,SAN LUIS GRANDE,61203,612,61203,-7.159046,-78.873084
...,...,...,...,...,...,...,...,...,...,...,...,...
1644,6,CAJAMARCA,3,CELENDIN,8,OXAMARCA,OXAMARCA,60308,603,60308,-7.059289,-78.061333
1645,6,CAJAMARCA,3,CELENDIN,9,SOROCHUCO,SOROCHUCO,60309,603,60309,-6.934215,-78.297182
1743,6,CAJAMARCA,8,JAEN,10,SAN FELIPE,SAN FELIPE,60810,608,60810,-5.786477,-79.310211
1744,6,CAJAMARCA,8,JAEN,6,LAS PIRIAS,LAS PIRIAS,60806,608,60806,-5.636342,-78.849590


In [10]:
table4 = table3[['UBIGEO', 'Centroid_Latitude', 'Centroid_Longitude']]
table4

,UBIGEO,Centroid_Latitude,Centroid_Longitude
0,60504,-7.364304,-78.917689
1,61105,-7.038039,-79.003318
2,61109,-6.926718,-79.131435
3,61110,-7.117978,-79.160968
4,61203,-7.159046,-78.873084
...,...,...,...
1644,60308,-7.059289,-78.061333
1645,60309,-6.934215,-78.297182
1743,60810,-5.786477,-79.310211
1744,60806,-5.636342,-78.849590


### The closest health establishment to Cajamarca district centroids

In [23]:
table5 =table1.merge(table4, how='left', left_on=['UBIGEO'], right_on=['UBIGEO'])
table5

,id_eess,codigo_renaes,categoria,nombre,diresa,red,direccion,longitud,latitud,id_ubigeo,...,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,CNT_CCPP,DESCRIPCIO,Centroid_Latitude,Centroid_Longitude
0,9439,4210,II-1,GRAL. JAEN,CAJAMARCA,NO PERTENECE A NINGUNA RED,AVENIDA AV. PAKAMAUROS S/N S/N AV. PAKAMAUROS ...,-78.804711,-5.706214,636,...,1,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551
1,27242,4211,I-4,MORRO SOLAR,CAJAMARCA,JAEN,OTROS ALFREDO BASTOS Nº 630 ALFREDO BASTOS Nº ...,-78.811920,-5.710600,636,...,1,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551
2,22911,4212,I-3,MAGLLANAL,CAJAMARCA,JAEN,CALLE CALLE LAS ALMENDRA CDRA. 4 S/N S/N CALLE...,-78.817266,-5.707647,636,...,1,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551
3,9242,4213,I-3,FILA ALTA,CAJAMARCA,JAEN,OTROS COMUNIDAD FILA ALTA COMUNIDAD FILA ALTA ...,-78.791643,-5.739184,636,...,1,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551
4,26418,4214,I-1,LAS NARANJAS,CAJAMARCA,JAEN,OTROS COMUNIDAD DE LAS NARANJAS COMUNIDAD DE L...,-78.846347,-5.745295,636,...,1,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,21625,5038,I-3,SAN LUIS DE LA LUCMA,CAJAMARCA,SOCOTA,CARRETERA CARRETERA LUCMA SOCOTA DISTRITO SAN ...,-78.607471,-6.292044,627,...,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753
1188,8179,6853,I-1,SAIREPAMPA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SAIREPAMPA DISTRITO SAN LUI...,-78.631080,-6.279960,627,...,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753
1189,16451,6870,I-1,SANTA ROSA DEL TINGO,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SANTA ROSA DEL TINGO DISTRI...,-78.597640,-6.266560,627,...,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753
1190,3040,7048,I-1,SEXESHITA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SEXESHITA DISTRITO SAN LUIS...,-78.627210,-6.267210,627,...,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753


In [24]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km



In [25]:
table5['Eucl_distance'] = [haversine(table5.longitud[i],table5.latitud[i],table5.Centroid_Latitude[i],table5.Centroid_Longitude[i]) for i in range(len(table5))]
table5

,id_eess,codigo_renaes,categoria,nombre,diresa,red,direccion,longitud,latitud,id_ubigeo,...,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,CNT_CCPP,DESCRIPCIO,Centroid_Latitude,Centroid_Longitude,Eucl_distance
0,9439,4210,II-1,GRAL. JAEN,CAJAMARCA,NO PERTENECE A NINGUNA RED,AVENIDA AV. PAKAMAUROS S/N S/N AV. PAKAMAUROS ...,-78.804711,-5.706214,636,...,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551,9024.511812
1,27242,4211,I-4,MORRO SOLAR,CAJAMARCA,JAEN,OTROS ALFREDO BASTOS Nº 630 ALFREDO BASTOS Nº ...,-78.811920,-5.710600,636,...,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551,9024.182044
2,22911,4212,I-3,MAGLLANAL,CAJAMARCA,JAEN,CALLE CALLE LAS ALMENDRA CDRA. 4 S/N S/N CALLE...,-78.817266,-5.707647,636,...,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551,9024.615245
3,9242,4213,I-3,FILA ALTA,CAJAMARCA,JAEN,OTROS COMUNIDAD FILA ALTA COMUNIDAD FILA ALTA ...,-78.791643,-5.739184,636,...,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551,9020.639943
4,26418,4214,I-1,LAS NARANJAS,CAJAMARCA,JAEN,OTROS COMUNIDAD DE LAS NARANJAS COMUNIDAD DE L...,-78.846347,-5.745295,636,...,JAEN,JAEN,60801,608,60801,108,JAEN,-5.770179,-78.852551,9021.105276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,21625,5038,I-3,SAN LUIS DE LA LUCMA,CAJAMARCA,SOCOTA,CARRETERA CARRETERA LUCMA SOCOTA DISTRITO SAN ...,-78.607471,-6.292044,627,...,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753,8937.444358
1188,8179,6853,I-1,SAIREPAMPA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SAIREPAMPA DISTRITO SAN LUI...,-78.631080,-6.279960,627,...,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753,8939.262579
1189,16451,6870,I-1,SANTA ROSA DEL TINGO,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SANTA ROSA DEL TINGO DISTRI...,-78.597640,-6.266560,627,...,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753,8940.017747
1190,3040,7048,I-1,SEXESHITA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SEXESHITA DISTRITO SAN LUIS...,-78.627210,-6.267210,627,...,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA,-6.279101,-78.578753,8940.572246
